<h1>SKLearn Tutorial: DNN on Boston Data</h1>
<br />
This tutorial follows very closely two other good tutorials and merges elements from both:
<ul>
<li><a href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/skflow/boston.py">https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/skflow/boston.py</a> 
<li><a href="http://bigdataexaminer.com/uncategorized/how-to-run-linear-regression-in-python-scikit-learn/">http://bigdataexaminer.com/uncategorized/how-to-run-linear-regression-in-python-scikit-learn/</a>

</ul>
<br />
D. Thiebaut<br />
August 2016.
<br />


<h2>Get the Boston Data</h2>
This part is basically taken directly from the <a href="http://bigdataexaminer.com/uncategorized/how-to-run-linear-regression-in-python-scikit-learn/">bigdataexaminer</a> tutorial.

<br />
Imports, first!


In [201]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from sklearn import cross_validation
from sklearn import metrics
from sklearn import preprocessing
import tensorflow as tf
#from tensorflow.contrib import learn
from tensorflow.contrib import learn
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import explained_variance_score,mean_absolute_error,median_absolute_error
from sklearn import metrics


Get the data...

In [225]:
street = pd.read_csv('Datatotalx.csv')

<h2>Convert the boston data into a panda data-frame</h2>


Add column names

In [226]:
x_data = street.drop('y',axis=1)
y_val = street['y']
from sklearn.model_selection import train_test_split

In [227]:
col_train = list(street.columns)
col_train_bis = list(street.columns)
col_train_bis.remove('y')

Split the data into training and test data.

In [228]:
feature_col=list(x_data.columns)
FEATURES=feature_col
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

In [229]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=43)

In [230]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [231]:
 scaler.fit( X_train )


MinMaxScaler(copy=True, feature_range=(0, 1))

Scale the X data to 0 mean and unit standard deviation

In [232]:
 scaler.fit( X_test )

MinMaxScaler(copy=True, feature_range=(0, 1))

X_train=np.matrix(X_train)
y_train=np.array(y_train)
X_test=np.matrix(X_test)
y_test=np.array(y_test)
y_train=y_train.reshape(-1,y_train.size)
y_test=y_test.reshape(-1,y_test.size)

<h1>Building a [10,10] Neural Net</h1>
<br />

In [233]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [234]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

In [235]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

In [239]:
opti = tf.train.AdamOptimizer(learning_rate = 0.001)

In [240]:
# Build a 2 layer fully connected DNN

regressor=tf.contrib.learn.DNNRegressor(hidden_units=[11,11],feature_columns=feature_cols,optimizer=opti, dropout=0.2)
regressor.fit( X_train, y_train, steps=5000, batch_size=1 )


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000027815E4C358>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\hp\\AppData\\Local\\Temp\\tmpj04vlaih'}


KeyError: 'x1'

Predict and score

In [ ]:
 y_predicted = regressor.predict( scaler.transform( X_test ) )
 score = metrics.mean_squared_error( y_predicted, y_test )
 print('MSE: {0:f}'.format(score) )


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter( y_predicted, y_test, s=5 )
plt.xlabel( "Predicted Prices")
plt.ylabel( "Real Prices")
plt.title( "Real vs Predicted Housing Prices")

<h1>Generating the graph of the NN with tensorboard</h1>
<br />
In order to generate the graph of the network, we simply add "model_dir = '/tmp/tf' " to the DNNRegressor() instantiation:




In [ ]:
'''
regressor = learn.DNNRegressor( feature_columns=None,
                                hidden_units=[10, 10] ),
                                model_dir = '/tmp/tf')
'''

This might generate an error when we try to fit, but that will happen after the useful information is written in /tmp/tf, and we can safely run tensorboard as follows, from the command line:

<pre>
tensorboard --logdir /tmp/tf
</pre>
<br />
And here's the graph we get for the 10x10 NN:
<br />
<center>
<img src="files/DNNRegressor1.png">
</center>

<h1>Building a [13,13] Neural Net</h1>
<br />

In [ ]:
# Build a 2 layer fully connected DNN
regressor = learn.DNNRegressor( feature_columns=None,
                               hidden_units=[13, 13])
regressor.fit(X_train, y_train, steps=5000, batch_size=10)


Predict and score

In [ ]:
 y_test_predicted = regressor.predict( scaler.transform( X_test ) )
 scoreTest = metrics.mean_squared_error( y_test_predicted, y_test )
 print('MSE Test Data: {0:f}'.format(scoreTest) )

 y_train_predicted = regressor.predict( X_train  )
 scoreTrain = metrics.mean_squared_error( y_train_predicted, y_train )
 print('MSE Train Data: {0:f}'.format(scoreTrain ) )


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter( y_predicted, y_test, s=5 )
plt.xlabel( "Predicted Prices")
plt.ylabel( "Real Prices")
plt.title( "Real vs Predicted Housing Prices")

<h1>Building a [13,13,10] Neural Net</h1>
<br />

In [ ]:
# Build a 2 layer fully connected DNN
feature_columns = boston.feature_names
regressor = learn.DNNRegressor( feature_columns=None,
                               hidden_units=[13, 13, 10],
                              model_dir = '/tmp/tf/')
regressor.fit(X_train, y_train, steps=5000, batch_size=1)


Predict and score

In [ ]:
 y_test_predicted = regressor.predict( scaler.transform( X_test ) )
 scoreTest = metrics.mean_squared_error( y_test_predicted, y_test )
 print('MSE Test Data: {0:f}'.format(scoreTest) )

 y_train_predicted = regressor.predict( X_train  )
 scoreTrain = metrics.mean_squared_error( y_train_predicted, y_train )
 print('MSE Train Data: {0:f}'.format(scoreTrain ) )


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter( y_predicted, y_test, s=5 )
plt.xlabel( "Predicted Prices")
plt.ylabel( "Real Prices")
plt.title( "Real vs Predicted Housing Prices")

<h1>Plotting Residuals</h1>

In [ ]:
plt.scatter( y_train_predicted, y_train_predicted - y_train,
           c ='b', s=30, alpha=0.4 )
plt.scatter(y_test_predicted, y_test_predicted - y_test,
           c ='g', s=30 )
plt.hlines( y=0, xmin=-5, xmax=55)
plt.title( "Residuals" )
plt.ylabel( "Residuals" )